# Installing Semantic Kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.18.0-alpha"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    // .AddOpenAIChatCompletion(
    //     modelId: Secrets.OpenAIModel,
    //     apiKey: Secrets.OpenAIApiKey)
    .Build();

### Import Bing search plugin

In [ ]:
#pragma warning disable SKEXP0050

var bingConnector = new BingConnector(Secrets.BingSearchKey);
var bing = new WebSearchEnginePlugin(bingConnector);

kernel.ImportPluginFromObject(bing, "bing");

### 📋 Add some arguments. Prepare Bing search function. 

In [ ]:
var promptTemplate = "When is the next total solar eclipse?";

var searchBingFunction = kernel.Plugins["bing"]["search"];

var kernelArguments = new KernelArguments()
{
    ["query"] = promptTemplate
};

var bingResult = await kernel.InvokeAsync(searchBingFunction, kernelArguments);

var currentContext = bingResult.GetValue<string>();

display(currentContext)


### 🏃‍♀️ Run prompt

In [ ]:
var promptTemplate = """
    When is the next total {{$input}} eclipse?

    ### real-time search result context
    {{$currentContext}}

    ### output format
    Respond only with JSON.
    """;

var executionSettings = new OpenAIPromptExecutionSettings 
{
     MaxTokens = 2000,
     Temperature = 0.1 
};

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate, executionSettings);

var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "solar",
    ["currentContext"] = currentContext
};

var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>());

### 👀 See rendered prompt

In [ ]:
#pragma warning disable SKEXP0001

display(response.RenderedPrompt);